In [1]:
#Imports
import numpy as np
import tensorflow as tf
# Model/layer imports
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [2]:
# #If using GPU, reset memory allocation, and assign GPU with memory growth.
# tf.compat.v1.reset_default_graph()
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
#Open Shakespeare.txt
path_to_file = "shakesbot.txt"
text = open(path_to_file, 'r').read()

In [4]:
#Obtain unique vocab set and size 
vocab = sorted(set(text))
vocab_size = len(vocab)

In [5]:
#Create an index mapping for characters in vocab (bi-directional)
char_to_ind = {char:ind for ind, char in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [6]:
#Encode whole of text file using vocab index map
encoded_text = np.array([char_to_ind[c] for c in text])

In [7]:
# #Query line(s) length(s)
# line = 'From fairest creatures we desire increase'
# print(len(line))

# lines='''
#   From fairest creatures we desire increase,
#   That thereby beauty's rose might never die,
#   But as the riper should by time decease,
#   His tender heir might bear his memory:
#   '''
# print(len(lines))

In [8]:
#Set length of sequence
seq_len = 150

In [9]:
#Determine total number of sequences in text
total_num_seq = len(text) // (seq_len+1)

In [10]:
#Create a Character Dataset from encoded text 
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [11]:
#Combine into sequences
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [12]:
# Create method for splitting sequences into input/target 
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

#Use method to create complete training dataset
dataset = sequences.map(create_seq_targets)

In [13]:
#Set batch size
batch_size = 64

In [14]:
#Shuffle the dataset
buffer_size=10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [15]:
#Set embedding dimensions
embed_dim = 256

In [16]:
#Choose amount of neurons in LSTM layer
rnn_neurons = 1024

In [17]:
#Customised Sparse_cat_cross loss function 
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [18]:
#Function for creating model
def create_model(vocab_size, embed_dim,rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size,None]))
    model.add(LSTM(rnn_neurons, stateful=True, return_sequences=True, recurrent_initializer='glorot_uniform', batch_input_shape=[batch_size,None, embed_dim]))
    model.add(Dropout(0.2))
    model.add(Dense(vocab_size))
    model.compile('adam', loss=sparse_cat_loss)
    return model

In [19]:
# Create training model
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [20]:
# Display model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           21504     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dropout (Dropout)            (64, None, 1024)          0         
_________________________________________________________________
dense (Dense)                (64, None, 84)            86100     
Total params: 5,354,580
Trainable params: 5,354,580
Non-trainable params: 0
_________________________________________________________________


In [21]:
#Set epochs 
epochs = 100

In [22]:
#Create early stopping function
early_stop = EarlyStopping(monitor='loss', patience=3)

In [23]:
#Fit the model
model.fit(dataset,epochs=epochs,callbacks=[early_stop])

Epoch 1/100
563/563 [==============================] - 35s 62ms/step - loss: 2.0706
Epoch 2/100
563/563 [==============================] - 35s 62ms/step - loss: 1.4518
Epoch 3/100
563/563 [==============================] - 35s 62ms/step - loss: 1.3117
Epoch 4/100
563/563 [==============================] - 35s 63ms/step - loss: 1.2466
Epoch 5/100
563/563 [==============================] - 35s 63ms/step - loss: 1.2047
Epoch 6/100
563/563 [==============================] - 35s 63ms/step - loss: 1.1748
Epoch 7/100
563/563 [==============================] - 35s 63ms/step - loss: 1.1512
Epoch 8/100
563/563 [==============================] - 36s 65ms/step - loss: 1.1312
Epoch 9/100
563/563 [==============================] - 35s 63ms/step - loss: 1.1137
Epoch 10/100
563/563 [==============================] - 36s 64ms/step - loss: 1.0987
Epoch 11/100
563/563 [==============================] - 37s 66ms/step - loss: 1.0844
Epoch 12/100
563/563 [==============================] - 36s 64ms/step - lo

In [24]:
#Save the model
model.save('shakesbot_train.h5')

In [25]:
#Create a new version of the model (test_model) with a single input batch size
test_model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
test_model.load_weights('shakesbot_train.h5')
test_model.build(tf.TensorShape([1,None]))
test_model.save('shakesbot_test.h5')

In [26]:
# Display test model summary
test_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86100     
Total params: 5,354,580
Trainable params: 5,354,580
Non-trainable params: 0
_________________________________________________________________


In [ ]:
###RUN THE BELOW CODE IF YOU ARE OPENING THIS AFTER TRAINING, AND ALREADY HAVE A PRE-TRAINED .h5 FILE###

In [ ]:
# #If loading seperately
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import numpy as np

# # #if using GPU, set the correct GPU and assign memory growth
# # tf.compat.v1.reset_default_graph()
# # physical_devices = tf.config.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# #Open shakesbot.txt
# path_to_file = "shakesbot.txt"
# text = open(path_to_file, 'r').read()

# #Obtain unique vocab set and size 
# vocab = sorted(set(text))
# vocab_size = len(vocab)

# #Create an index mapping for characters in vocab (bi-directional)
# char_to_ind = {char:ind for ind, char in enumerate(vocab)}
# ind_to_char = np.array(vocab)

# #Load the model to memory
# test_model = load_model('shakesbot_test.h5', compile=False)

In [ ]:
###RUN THE ABOVE CODE IF YOU ARE OPENING THIS AFTER TRAINING, AND ALREADY HAVE A PRE-TRAINED .h5 FILE###

In [27]:
#Test function for generating consecutive text
def generate_text(model,start_seed,num_generate=500,temperature=1):
    
    #Map each character in start_seed to it's relative index
    input_eval = [char_to_ind[s] for s in start_seed]
    
    #Expand the dimensions
    input_eval = tf.expand_dims(input_eval, 0)
    
    #Create empty list for generated text
    text_generated = []
    
    #Reset the states of tyhe model        
    model.reset_states()
    
    #for each iteration of num_generate
    for i in range(num_generate):
        
        #Obtain probability matrix for current iteration 
        predictions = model(input_eval)
        
        #Reduce dimensions
        predictions = tf.squeeze(predictions,0)
        
        #Multiply probabily matrix by temperature
        predictions = predictions/temperature
        
        #Select a random outcome, based on the unnormalised log-probabilities produced by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        #Expand dimensions of prediction and assign as next input evaluation
        input_eval = tf.expand_dims([predicted_id], 0)
        
        #Convert prediction to char and append to generated list of text
        text_generated.append(ind_to_char[predicted_id])
        
    #return the initial input, concatenated with the generated text. 
    return(start_seed+"".join(text_generated))

In [28]:
# Test the model with sample text
print(generate_text(model=test_model,start_seed="This is a test", num_generate=500, temperature=0.5))

This is a testimony that the world should love his own precious breath.
    The husband that hath struck the same for                                                        Exit
  FLUTE. There is no worse a kind of princely father
    That should revenge the heavy time of thine.
    Then when thou speak'st not not which thou wilt fear
    The flattering of thy life I should suffer
    A present that I thought he shall be sold.
    Then, to thy spirit, dear to thee, are here.
                                  
